In [1]:
import os

In [2]:
%pwd

'/Users/main/Desktop/LSTM_Forecast/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/main/Desktop/LSTM_Forecast'

In [11]:
import pandas as pd
import numpy as np
from TimeSeriesForecast import logger
from TimeSeriesForecast.utils.common import read_yaml,create_directories
from TimeSeriesForecast.constants import *
from dataclasses import dataclass
from pathlib import Path
import torch
from notebooks.src.functions_torch import TimeSeries
from notebooks.src.functions_torch import LSTM
from notebooks.src.functions_torch import set_seed
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [8]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    save_path: Path

In [10]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH,
                 schema_filepath= SCHEMA_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def model_trainer_config(self)->ModelTrainerConfig:
        config= self.config.model_trainer
        
        create_directories([config.root_dir])
        
        model_trainer_config= ModelTrainerConfig(
                                                root_dir= self.root_dir,
                                                data_path= self.data_path,
                                                save_path= self.data_path
                                                )
        return model_trainer_config

In [ ]:

class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config= config
        self.tensor= None
        self.X_train= None
        self.X_test= None
        self.y_train= None
        self.y_test= None
        
    def tensor_loader(self,seed:42):
        self.seed=seed
        torch.manual_seed(self.seed)
        torch.cuda.manual_seed(self.seed)
        np.random.seed(self.seed)
        try:
            self.tensor= torch.load(self.config.data_path)
            self.X_train,self.X_test,self.y_train,self.y_test = self.tensor['X_train'],self.tensor['X_test'],\
                                                                self.tensor['y_train'],self.tensor['y_test']
            return self
        except Exception as e:
            raise Exception(f'Problem loading the tensors: {e}')
    def time_series_dataset(self):
        batch_size=16
        try:
            self.train_dataset= TimeSeries(self.X_train,self.y_train)
            self.test_dataset= TimeSeries(self.X_test,self.y_test)
            
            self.train_loader= DataLoader(self.train_dataset,batch_size=batch_size,shuffle=True)
            self.test_loader= DataLoader(self.test_dataset,batch_size=batch_size,shuffle=True)
        except Exception as e:
            raise Exception(f'Problem the class modules {e}')
        return self
    

    
    
                                                            
        